![nebullvm nebuly AI accelerate inference optimize DeepLearning](https://user-images.githubusercontent.com/38586138/201391643-a80407e5-2c28-409c-90c9-327795cd27e8.png)

# Accelerate PyTorch ResNet50 with AutoBoost

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using AutoBoost app from the open-source library `nebullvm`.

We will
1. Install AutoBoost and the deep learning compilers used by the library.
2. Speed up a PyTorch ResNet50 without any loss of accuracy.
3. Achieve faster acceleration on the same model by applying more aggressive optimization techniques (e.g. pruning, quantization) under the constraint of sacrificing up to 2% accuracy.

Let's jump to the code.

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


### Installation

In [ ]:
!pip install autoboost

This is an optional step. Run it if you want to contribute to continuous improvement of `nebullvm` and share the performance achieved with it. You can find full details in the [docs](https://nebuly.gitbook.io/nebuly/nebullvm/how-nebullvm-works/fostering-continuous-improvement#sharing-feedback-to-improve-nebullvm).

In [1]:
json_feedback = {
    "allow_feedback_collection": True
}
import json
from pathlib import Path

(Path.home() / ".nebullvm").mkdir(exist_ok=True)
with open(Path.home() / ".nebullvm/collect.json", "w") as f:
  json.dump(json_feedback, f)

Let's now import install the deep learning compilers used by AutoBoost that are not yet installed on the hardware.

The installation of the compilers may take a few minutes.

In [ ]:
!python -m nebullvm.installers.auto_installer  --frameworks torch onnx --compilers all

## Optimization example with Pytorch

In the following example we will try to optimize a standard resnet50 loaded directly from torchvision.

AutoBoost can accelerate neural networks without loss of a user-defined precision metric, e.g. accuracy, or can achieve faster acceleration by applying more aggressive optimization techniques, such as pruning and quantization, that may have a negative impact on the selectic metric. The maximum threshold value for accuracy loss is determined by the metric_drop_ths parameter. Read more in the [docs](https://nebuly.gitbook.io/nebuly/nebullvm/get-started).

Let first test the optimization without accuracy loss (metric_drop_ths=0, default value), and then apply further accelerate it under the constrained of losing up to 2% of accuracy (metric = "accuracy", metric_drop_ths = 0.02).

### Scenario 1 - No accuracy drop

First we load the model and optimize it using the AutoBoost API:

In [ ]:
import torch
import torchvision.models as models
from autoboost import optimize_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a resnet as example
model = models.resnet50().to(device)

# Provide an input data for the model    
input_data = [((torch.randn(1, 3, 256, 256), ), torch.tensor([0]))]

# Run AutoBoost optimization
optimized_model = optimize_model(
  model, input_data=input_data, optimization_time="unconstrained"
)

# Try the optimized model
x = torch.randn(1, 3, 256, 256).to(device)
model.eval()
res_optimized = optimized_model(x)
res_original = model(x)

We can print the type of the optimized model to see which compiler was faster:

In [5]:
optimized_model

PytorchTensorRTInferenceLearner(network_parameters=ModelParams(batch_size=1, input_infos=[<nebullvm.base.InputInfo object at 0x7f0d6cbac2d0>], output_sizes=[(1000,)], dynamic_info=None), input_tfms=<nebullvm.transformations.base.MultiStageTransformation object at 0x7f0dc9159c10>)

In our case, the optimized model type was PytorchTensorRTInferenceLearner, so this means that Pytorch-TensorRT was the faster compiler.

After the optimization step, we can compare the optimized model with the baseline one in order to verify that the output is the same and to measure the speed improvement

First of all, let's print the results

In [ ]:
res_original

In [ ]:
res_optimized

Then, let's compare the performances:

In [8]:
from nebullvm.utils.benchmark import benchmark

In [ ]:
# Set the model to eval mode and move it to the available device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.eval()
model.to(device)

Here we compute the average throughput for the baseline model:

In [10]:
benchmark(model, input_data)

Performing benchmark on 1000 iterations: 100%|██████████| 1000/1000 [00:09<00:00, 102.48it/s]

Input shapes: [torch.Size([1, 3, 256, 256])]
Output features shapes: [torch.Size([1000])]
Batch size: 1
Average Throughput: 104.05 data/second
Average Latency: 0.0096 seconds/data


Here we compute the average throughput for the optimized model:



In [11]:
benchmark(optimized_model, input_data)

Performing benchmark on 1000 iterations: 100%|██████████| 1000/1000 [00:03<00:00, 269.16it/s]

Input shapes: [torch.Size([1, 3, 256, 256])]
Output features shapes: [torch.Size([1000])]
Batch size: 1
Average Throughput: 274.01 data/second
Average Latency: 0.0036 seconds/data


## Scenario 2 - Accuracy drop

In this scenario, we set a max threshold for the accuracy drop to 2%

In [ ]:
import torch
import torchvision.models as models
from autoboost import optimize_model

# Load a resnet as example
model = models.resnet50().to(device)

# Provide 100 random input data for the model  
input_data = [((torch.randn(1, 3, 256, 256), ), torch.tensor([0])) for _ in range(100)]

# Run AutoBoost optimization
optimized_model = optimize_model(
  model, input_data=input_data, optimization_time="unconstrained", metric="accuracy", metric_drop_ths=0.02
)

# Try the optimized model
x = torch.randn(1, 3, 256, 256).to(device)
res = optimized_model(x)

In [ ]:
# Set the model to eval mode and move it to the available device

model.eval()
model.to(device)

Here we compute the average throughput for the baseline model:

In [17]:
benchmark(model, input_data)

Performing benchmark on 1000 iterations: 100%|██████████| 1000/1000 [00:09<00:00, 102.56it/s]

Input shapes: [torch.Size([1, 3, 256, 256])]
Output features shapes: [torch.Size([1000])]
Batch size: 1
Average Throughput: 104.03 data/second
Average Latency: 0.0096 seconds/data


Here we compute the average throughput for the optimized model:

In [18]:
benchmark(optimized_model, input_data)

Performing benchmark on 1000 iterations: 100%|██████████| 1000/1000 [00:01<00:00, 937.67it/s]

Input shapes: [torch.Size([1, 3, 256, 256])]
Output features shapes: [torch.Size([1000])]
Batch size: 1
Average Throughput: 960.64 data/second
Average Latency: 0.0010 seconds/data


<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Join the community </a> |
    <a href="https://nebuly.gitbook.io/nebuly/welcome/questions-and-contributions" target="_blank" style="text-decoration: none;"> Contribute to the library </a>
</center>

<center> 
    <a href="https://github.com/nebuly-ai/nebullvm#how-it-works" target="_blank" style="text-decoration: none;"> How nebullvm works </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#documentation" target="_blank" style="text-decoration: none;"> Documentation </a> •
    <a href="https://github.com/nebuly-ai/nebullvm#api-quick-view" target="_blank" style="text-decoration: none;"> API quick view </a> 
</center>